In [1]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# find_dvspg_by_nameメソッド

```python
def find_dvspg_by_name(dv_switch, portgroup_name)
```

vimtypeを指定してManaged Objectを検索するメソッドです。  

## 引数

| args           | type                     | param                             |
| :-             | :----------------------- | :-------------------------------- |
| dvswitch       | class                    | 分散スイッチのManaged Object      |
| portgroup_name | str                      | 検索するポートグループ名          |

## 戻り値

| rtype             | return                                                                                      |
| :---------------- | :------------------------------------------------------------------------------------------ |
| class             | 分散ポートグループが存在した場合は分散ポートグループのManaged Objedt                        |
| book              | 分散ポートグループが存在しない場合はNone                                                    |

# find_dvspg_by_nameメソッド動作確認

find_dvspg_by_nameメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* dvs_name
* pg_name
* hostname
* username
* password
* validate_certs

戻り値はfirstがTrueの場合とFalseで違いを確認してみます。

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_dvs_by_name, find_dvspg_by_name
from pyVmomi import vim

dvs_name = 'dvs Name'
pg_name = 'PG Name'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
dvs_obj = find_dvs_by_name(sc, dvs_name)
if(dvs):
    pg_obj = find_dvspg_by_name(dvs_obj, pg_name)
    if(pg_obj):
        print(pg_obj.name)
    else:
        print("%s not found" % pg_name)
else:
    print("%s not found" % dvs_name)

# find_dvspg_by_nameを使ったモジュール例

find_dvspg_by_nameを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはVM名をしてフィルタリングされた結果を戻すだけの単純なものです。  
モジュールは `/tmp/ansible_salf_made_module` に `get_pg_name.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/get_pg_name.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import find_dvs_by_name, find_dvspg_by_name, connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()
    argument_spec.update(dvs_name=dict(type='str', required=True),
                         pg_name=dict(type='str', required=True))

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    dvs_name = module.params['dvs_name']
    pg_name = module.params['pg_name']

    result = dict(changed=False)
    sc = connect_to_api(module)
    dvs_obj = find_dvs_by_name(sc, dvs_name)
    if(dvs_obj):
        pg_obj = find_dvspg_by_name(dvs_obj, pg_name)
        if(pg_obj):
            result['pg_name'] = pg_obj.name
            module.exit_json(**result)
        else:
            module.fail_json(msg="%s not found" % pg_name)
    else:
        module.fail_json(msg="%s not found" % dvs_name)

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs
* name

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
get_pg_name:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no
  dvs_name: TEST
  pg_name: DPortGroup

`pg_name` に指定したポートグループ名が取得できていることを確認します。